# Libs

In [19]:
from psw import MyClass # access information
import openai # openai lib
import tiktoken # lib used by openai the tokenize words

# Main Variables

In [20]:
access = MyClass()
openai.api_key = access.key
model = access.model['model']['name']
max_tokens = access.model['model']['limit_tokens']

# User message

In [195]:
def input_text():
    text = input("user: ")
    text_transformed = [{'content': text}]
    return text, text_transformed

In [196]:
text, text_transformed = input_text()

In [198]:
print(text)
print(text_transformed)

what is the scape velocity of mars orbit?
[{'content': 'what is the scape velocity of mars orbit?'}]


# Tokens

In [199]:
encoding = tiktoken.encoding_for_model(model)

In [200]:
encoding

<Encoding 'cl100k_base'>

In [201]:
text_encoded = encoding.encode(text)
number_tokens = len(text_encoded)

In [202]:
print(f'Text encoded: {text_encoded}')
print(f'Number of tokens: {number_tokens}')

Text encoded: [12840, 374, 279, 99113, 15798, 315, 48449, 27605, 30]
Number of tokens: 9


In [203]:
encoding.decode(text_encoded)

'what is the scape velocity of mars orbit?'

In [204]:
type(text)

str

In [207]:
def input_counter(messages, model=model):
    # messages = [{'content': messages}] # mudar nome da variavel
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [208]:
input_counter(text_transformed, model)

15

In [59]:
response = openai.ChatCompletion.create(
  model=model,
  messages=[{"role": "system", "content" : message}],
  max_tokens=100
)

print(response)


TypeError: Object of type set is not JSON serializable

In [36]:
text_encoded = encoding.encode(response['choices'][0]['message']['content'])
number_tokens = len(text_encoded)

In [37]:
print(f'Text encoded: {text_encoded}')
print(f'Number of tokens: {number_tokens}')

Text encoded: [791, 12731, 15798, 315, 21725, 6, 27605, 374, 13489, 220, 20, 13, 2839, 41668, 824, 2132, 11, 477, 220, 806, 11, 9714, 8931, 824, 6596, 13, 1115, 3445, 430, 459, 1665, 1053, 1205, 311, 5662, 420, 4732, 311, 23075, 279, 71019, 6958, 315, 21725, 323, 12731, 1202, 27605, 13]
Number of tokens: 49


In [27]:
response.created

1691459089

In [29]:
response.choices[0].message['content'].strip()

"The escape velocity of Mars' orbit is approximately 5.03 kilometers per second, or about 3.13 miles per second."

In [11]:
response.usage

<OpenAIObject at 0x177b0b2e6b0> JSON: {
  "prompt_tokens": 10,
  "completion_tokens": 17,
  "total_tokens": 27
}

In [ ]:
def total_tokens(response):
    total_tokens = response.usage['total_tokens']
    creation = response.created
    
    return total_tokens, creation